# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



#### 1 Each column's  missing value counts

In [1]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
import pandas as pd
crunch_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')

missing = []

for chunk in crunch_iter:
    missing.append(chunk.isna().sum())

    
combined_missing = pd.concat(missing)
combined_missing.groupby(combined_missing.index).sum().sort_values(ascending= False)

investor_category_code    50427
investor_state_code       16809
investor_city             12480
investor_country_code     12001
raised_amount_usd          3599
company_category_code       643
company_city                533
company_state_code          492
funded_at                     3
funded_month                  3
funded_quarter                3
funding_round_type            3
funded_year                   3
investor_name                 2
investor_permalink            2
investor_region               2
company_region                1
company_permalink             1
company_name                  1
company_country_code          1
dtype: int64

#### 2. Each columns Memory Foot print

In [2]:
crunch_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')
memory_used =  []

for chunk in crunch_iter:
      memory_used.append(chunk.memory_usage(deep= True)/2**20)


combined_memory = pd.concat(memory_used)
combined_memory.groupby(combined_memory.index).sum().sort_values(ascending= False)
        


investor_permalink        4.749821
company_permalink         3.869808
investor_name             3.734270
company_name              3.424955
funded_at                 3.378091
company_city              3.343473
company_category_code     3.262619
company_region            3.253503
funding_round_type        3.252704
investor_region           3.238946
funded_quarter            3.226837
funded_month              3.226837
company_country_code      3.025223
company_state_code        2.962161
investor_city             2.751430
investor_country_code     2.524654
investor_state_code       2.361876
investor_category_code    0.593590
raised_amount_usd         0.403366
funded_year               0.403366
Index                     0.001381
dtype: float64

### 3.Total foot print of memory chunks combined

In [3]:
total_memory=sum(combined_memory)
print(total_memory)

56.988911628723145


### 4.Which column(s) we can drop because they aren't useful for analysis.

In [4]:
to_drop = ['Index']


cols_to_use = [
    "company_name", "company_category_code", "company_country_code", "company_state_code","company_city",
    "investor_name", "investor_category_code", "investor_country_code", "investor_state_code", "investor_city",
    "funding_round_type", "funded_at", "raised_amount_usd","funded_month"
]

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


##### 2a The data type for each column

In [5]:
# The types for each column


crunch_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')
data_type =  []
count = 0 # Have introduced this counter to keep track of the number of chunks
for chunk in crunch_iter:
      data_type.append(chunk.dtypes)
      count += 1


print(f'number of chunks {count}')
combined_data_type = pd.concat(data_type)
combined_data_type.groupby(combined_data_type.index).value_counts()




number of chunks 11


company_category_code   object     11
company_city            object     11
company_country_code    object     11
company_name            object     11
company_permalink       object     11
company_region          object     11
company_state_code      object     11
funded_at               object     11
funded_month            object     11
funded_quarter          object     11
funded_year             int64      10
                        float64     1
funding_round_type      object     11
investor_category_code  float64    10
                        object      1
investor_city           object      9
                        float64     2
investor_country_code   object      9
                        float64     2
investor_name           object     11
investor_permalink      object     11
investor_region         object     11
investor_state_code     object      9
                        float64     2
raised_amount_usd       float64    11
dtype: int64

##### Observations:  



There were a total of  11 chunks with 14 and 2 columns as string and numeric datatypes respectively. 

There were 4 columns that had some chunks distributed between strings and numeric data types. These were investor_category_code,investor_city,investor_country_code and investor_state_code. 

The funded_year and  raised_amount_usd can be converted into appropriate subtypes to conserve space.
 

#### 2c Investigating the text columns

In [6]:
chunk_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')



uniques = {}

for chunk in chunk_iter:
    strs = chunk.select_dtypes(include=['object'])
    for col in strs.columns:
        chunk_vc = chunk[col].value_counts()
        if col in uniques:
         uniques[col].append(chunk_vc)
        uniques[col]=[chunk_vc]

uniques_combined = {}

for col in uniques:
    u_concat = pd.concat(uniques[col])
    u_group = u_concat.groupby(u_concat.index).sum()
    uniques_combined[col] = u_group
    if u_group.shape[0] < 50:
        print(col)


company_category_code
company_country_code
company_state_code
investor_category_code
investor_country_code
investor_state_code
investor_region
funding_round_type


##### Observations:  
company_category_code,company_country_code,company_state_code,investor_category_code,investor_country_code,investor_state_code,investor_region,and funding_round_type have unique values below 50% ad hence can be converted to numeric to save space.

Checking text columns that can be cleaned 

In [7]:
# We want to read a chunk of records and then view the head to see if there are any columns that can be cleaned further.
chunk_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')
records = []
for chunk in chunk_iter:
    text_cols = chunk.select_dtypes(include='object')
    records.append(text_cols.sample(1000)) # From the 5000 chunks being read, sample 1000 records to get a look of columns that can be cleaned.


final_df=pd.concat(records)
final_df[0:5]


,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter
2346,/company/otherinbox,OtherInbox,messaging,USA,TX,Austin,Austin,/company/source-spring,Source Spring,NaN,NaN,NaN,unknown,NaN,series-a,2010-09-20,2010-09,2010-Q3
2258,/company/dollar-shave-club,Dollar Shave Club,ecommerce,USA,CA,Los Angeles,Los Angeles,/company/science,Science,finance,USA,CA,Los Angeles,Santa Monica,angel,2012-03-06,2012-03,2012-Q1
2953,/company/appthwack,AppThwack,mobile,USA,OR,Portland,Portland,/company/upstart-labs,Upstart Labs,software,USA,OR,Portland,Portland,angel,2013-03-01,2013-03,2013-Q1
4826,/company/viewray,ViewRay,medical,USA,OH,Cleveland,Oakwood Village,/financial-organization/aisling-capital,Aisling Capital,NaN,USA,NY,New York,New York,series-c+,2013-05-21,2013-05,2013-Q2
3263,/company/kippt,Kippt,web,USA,CA,SF Bay,San Francisco,/company/y-combinator,Y Combinator,finance,USA,CA,SF Bay,Mountain View,angel,2012-06-01,2012-06,2012-Q2


Observations: Funded month can be separated into year and month numeric columns but its not necessary for this analysis

In [8]:
# Re writing the last step to ensure that overall memory stays below 10Mb
# Strategy is to reduce the samples to be below 1000 records for each chunk. This was arrived after experimentation
chunk_iter = pd.read_csv('crunchbase.csv', chunksize=5000,encoding='latin1')
records = []
memory_usage = []
for chunk in chunk_iter:
    text_cols = chunk.select_dtypes(include='object')
    samples = text_cols.sample(100)
    records.append(samples)
    memory_usage.append(samples.memory_usage(deep=True).sum()/2**20)


final_df=pd.concat(records)
count = 1
for i in memory_usage:
    print(f' Memory consumption for  chunk {count} : {i} Mbs')
    count += 1
print(f'Overall consumption is {sum(memory_usage)} Mbs')

 Memory consumption for  chunk 1 : 0.11028003692626953 Mbs
 Memory consumption for  chunk 2 : 0.10892486572265625 Mbs
 Memory consumption for  chunk 3 : 0.10893440246582031 Mbs
 Memory consumption for  chunk 4 : 0.10940074920654297 Mbs
 Memory consumption for  chunk 5 : 0.108856201171875 Mbs
 Memory consumption for  chunk 6 : 0.109222412109375 Mbs
 Memory consumption for  chunk 7 : 0.10869693756103516 Mbs
 Memory consumption for  chunk 8 : 0.10837554931640625 Mbs
 Memory consumption for  chunk 9 : 0.10758304595947266 Mbs
 Memory consumption for  chunk 10 : 0.08908748626708984 Mbs
 Memory consumption for  chunk 11 : 0.0891122817993164 Mbs
Overall consumption is 1.1584739685058594 Mbs


In [9]:
# The data types that will be used in loading the final chunks

col_types = {
    "company_name": "category", "company_category_code": "category", "company_country_code": "category",
    "company_state_code": "category", "company_city": "category", "investor_name": "category",
    "investor_category_code": "category", "investor_country_code": "category", "investor_state_code": "category",
    "investor_city": "category", "funding_round_type": "category", "raised_amount_usd": "float64"
}

## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [10]:
# Your code goes here

import sqlite3
import pandas as pd
conn = sqlite3.connect('chrunch.db')

chunk_iter = pd.read_csv('crunchbase.csv', chunksize=5000,usecols=cols_to_use,encoding='latin1',parse_dates=['funded_at'],dtype=col_types)


for chunk in chunk_iter:
 #drop the rows that are null
 chunk.dropna(inplace=True)

 #Change the raised_amount_usd to float
 chunk['raised_amount_usd']=chunk['raised_amount_usd'].astype('float')
 
 #Split the funded month and extract month
 chunk['month'] = chunk['funded_month'].str.split('-').str[-1]
 
 # Change the month column to integer to save space
 chunk['month']= pd.to_numeric(chunk['month'],downcast = 'signed')
 
 #Extract year from funded month
 chunk['year']=chunk['funded_month'].str.split('-').str[0]

 # Funded month column is no longer required since we have the month column 
 chunk.drop(['funded_month'], axis = 1,inplace=True)
 chunk.to_sql("chrunch", conn, if_exists='append', index=False)



In [11]:
# Quering the table to check that it has the right data types
results_df = pd.read_sql('PRAGMA table_info(chrunch);', conn)
print(results_df)

    cid                    name       type  notnull dflt_value  pk
0     0            company_name       TEXT        0       None   0
1     1   company_category_code       TEXT        0       None   0
2     2    company_country_code       TEXT        0       None   0
3     3      company_state_code       TEXT        0       None   0
4     4            company_city       TEXT        0       None   0
5     5           investor_name       TEXT        0       None   0
6     6  investor_category_code       TEXT        0       None   0
7     7   investor_country_code       TEXT        0       None   0
8     8     investor_state_code       TEXT        0       None   0
9     9           investor_city       TEXT        0       None   0
10   10      funding_round_type       TEXT        0       None   0
11   11               funded_at  TIMESTAMP        0       None   0
12   12       raised_amount_usd       REAL        0       None   0
13   13                   month    INTEGER        0       None

In [12]:
#Checking that we have read the complete data set
pd.read_sql('''SELECT * FROM chrunch;''', conn)

,company_name,company_category_code,company_country_code,company_state_code,company_city,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_city,funding_round_type,funded_at,raised_amount_usd,month,year
0,LaunchGram,news,USA,CA,Mountain View,10Xelerator,finance,USA,OH,Columbus,other,2012-01-23 00:00:00,20000.0,1,2012
1,ZoopShop,software,USA,OH,columbus,10Xelerator,finance,USA,OH,Columbus,angel,2012-02-15 00:00:00,20000.0,2,2012
2,eFuneral,web,USA,OH,Cleveland,10Xelerator,finance,USA,OH,Columbus,other,2011-09-08 00:00:00,20000.0,9,2011
3,Tackk,web,USA,OH,Cleveland,10Xelerator,finance,USA,OH,Columbus,other,2012-02-01 00:00:00,20000.0,2,2012
4,Acclaimd,analytics,USA,OH,Columbus,10Xelerator,finance,USA,OH,Columbus,angel,2012-06-01 00:00:00,20000.0,6,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,OpenTable,web,USA,CA,San Francisco,Zagat,web,USA,NY,New York City,series-b,2000-01-01 00:00:00,10000000.0,1,2000
1574,GainSpan,semiconductor,USA,CA,San Jose,Zebra Technologies,enterprise,USA,IL,Lincolnshire,series-c+,2013-06-26 00:00:00,19000000.0,6,2013
1575,Wheelz,automotive,USA,CA,San Francisco,Zipcar,transportation,USA,GA,NW Atlanta,series-a,2012-02-22 00:00:00,13700000.0,2,2012
1576,Red Tricycle,web,USA,CA,Sausalito,zulily,ecommerce,USA,WA,Seattle,series-a,2012-11-30 00:00:00,1500000.0,11,2012


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

Question 4a What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised

In [13]:
#top 10 percent and proportion raised
query='''
      SELECT
            top_10.company_name,
            sum(raised_amount_usd)/(select sum(raised_amount_usd) from chrunch) AS percentage_funding,
            sum(raised_amount_usd) AS funding_amount
      FROM 
           chrunch as top_10
      GROUP BY
             top_10.company_name 
      ORDER BY
            funding_amount DESC
      LIMIT
           (SELECT CAST(count(distinct company_name)*.1 AS INT) from chrunch)
      '''

In [14]:
top_10_raised=pd.read_sql(query,conn)
print("funding raised by top 10 percent %.2f billion dollars"%(top_10_raised["funding_amount"].sum()/10000000000))

funding raised by top 10 percent 3.82 billion dollars


In [15]:
#top 1 percent and proportion raised
query='''
      SELECT
            top_1.company_name,
            sum(raised_amount_usd)/(select sum(raised_amount_usd) from chrunch) AS percentage_funding,
            sum(raised_amount_usd) AS funding_amount
      FROM 
           chrunch as top_1
      GROUP BY
             top_1.company_name 
      ORDER BY
            funding_amount DESC
      LIMIT
           (SELECT CAST(count(distinct company_name)*.01 AS INT) from chrunch)
      '''

In [16]:
top_1_raised=pd.read_sql(query,conn)
print("funding raised by top 1 percent %.2f billion dollars"%(top_1_raised["funding_amount"].sum()/10000000000))

funding raised by top 1 percent 2.94 billion dollars


In [17]:
#Bottom 10 percent and proportion invested
query='''
      SELECT
            bottom_10.company_name,
            round(sum(raised_amount_usd)/(select sum(raised_amount_usd) from chrunch),6) AS percentage_funding,
            sum(raised_amount_usd) AS funding_amount
      FROM 
           chrunch as bottom_10
      GROUP BY
             bottom_10.company_name 
      HAVING 
            funding_amount IS NOT Null
      ORDER BY
            funding_amount ASC
      LIMIT
           (SELECT CAST(count(distinct company_name)*.1 AS INT) from chrunch)
      '''

In [18]:
bottom_10_raised=pd.read_sql(query,conn)
print("funding raised by bottom 10 percent %.2f million dollars"%(bottom_10_raised["funding_amount"].sum()/1000000))

funding raised by bottom 10 percent 1.96 million dollars


In [19]:
# Bottom 1 percent and proportion invested
query='''
      SELECT
            bottom_1.company_name,
            round(sum(raised_amount_usd)/(select sum(raised_amount_usd) from chrunch),6) AS percentage_funding,
            sum(raised_amount_usd) AS funding_amount
      FROM 
           chrunch as bottom_1
      GROUP BY
             bottom_1.company_name 
      HAVING 
            funding_amount IS NOT Null
      ORDER BY
            funding_amount ASC
      LIMIT
           (SELECT CAST(count(distinct company_name)*.01 AS INT) from chrunch)
        '''

In [20]:
bottom_1_raised=pd.read_sql(query,conn)
print("funding raised by bottom 10 percent %.2f million dollars"%(bottom_1_raised["funding_amount"].sum()/1000000))

funding raised by bottom 10 percent 0.12 million dollars


In [21]:
#Company category and invested amount
pd.read_sql('''
SELECT 
      company_category_code,
      SUM(raised_amount_usd) AS raised_amount
FROM 
      chrunch
GROUP BY 
      company_category_code
ORDER BY 
      raised_amount DESC
LIMIT 
      10

''',conn)

,company_category_code,raised_amount
0,mobile,2.619877e+10
1,software,2.901726e+09
2,biotech,2.744865e+09
3,web,2.032014e+09
4,enterprise,1.545868e+09
5,ecommerce,1.448791e+09
6,cleantech,1.258376e+09
7,advertising,1.253556e+09
8,games_video,8.353405e+08
9,hardware,8.056420e+08


Question:Which category of company attracted the most investments?

Answer: Mobile industry had the most investment.

In [22]:
pd.read_sql('''
SELECT 
      investor_name,
      SUM(raised_amount_usd) AS raised_amount
FROM 
      chrunch
GROUP BY 
      investor_name
ORDER BY 
      raised_amount DESC
LIMIT 10


''',conn)

,investor_name,raised_amount
0,Intel,5.969200e+09
1,Google,5.808800e+09
2,Time Warner,5.730000e+09
3,Comcast,5.669000e+09
4,BrightHouse,4.700000e+09
5,Cisco,1.061730e+09
6,Microsoft,1.037500e+09
7,Amazon,8.838000e+08
8,Amgen,5.314000e+08
9,General Electric,4.550000e+08


Question:Which investor contributed the most money (across all startups)?

Answer: Intel

In [23]:
pd.read_sql('''
SELECT 
      investor_name,
      COALESCE(SUM(raised_amount_usd),0) AS raised_amount,
      company_name
FROM 
      chrunch
GROUP BY 
      investor_name,
      company_name
ORDER BY 
      raised_amount DESC
LIMIT 
      10

''',conn)

,investor_name,raised_amount,company_name
0,Comcast,5.620000e+09,Clearwire
1,Intel,5.620000e+09,Clearwire
2,Time Warner,5.620000e+09,Clearwire
3,BrightHouse,4.700000e+09,Clearwire
4,Google,3.200000e+09,Clearwire
5,Google,1.000000e+09,AOL
6,Amazon,7.590000e+08,LivingSocial
7,Google,4.500000e+08,SurveyMonkey
8,A123 Systems,3.390000e+08,Fisker Automotive
9,EDS,3.000000e+08,Nanosolar


Question:Which investors contributed the most money per startup?

Answer: Comcast,Intel,and Time Warner

In [24]:
pd.read_sql('''
SELECT 
      count(funding_round_type) AS count,
      funding_round_type,
      SUM(raised_amount_usd) AS raised_amount
FROM 
      chrunch
GROUP BY 
      funding_round_type

ORDER BY 
      raised_amount DESC

''',conn)

,count,funding_round_type,raised_amount
0,13,post-ipo,2.576000e+10
1,283,series-c+,8.186735e+09
2,214,venture,5.485991e+09
3,224,series-b,3.725668e+09
4,272,series-a,1.874929e+09
5,12,private-equity,7.028977e+08
6,61,other,5.275765e+08
7,498,angel,1.523852e+08
8,1,crowdfunding,5.650000e+04


Question:Which funding round was the most popular? Which was the least popular?

Answer: The Most popular funding type was post-ipo, The least popular was crowdfunding

NB: The answer to this question changes to series-c+ if you dont remove duplicates during loading of data into sqlite